# Nucleolus Tracking Pipeline
This notebook contains all necessary steps to first relabel nuclei by giving each a unique ID over time, then the labeled segmented images for nucleoli and nuclei will be merged into a single file for each timelapse so that they can be visualized in FIJI. Once opened in FIJI, you will need to manually note down the ID of nuclei that interest you. These triplets will need to be entered into an excel. Finally, you will return to this notebook and run the two final cells where the distances will be computed.

<span style='color:red'> **1. Run the first 6 cells** </span>

In [ ]:
from itkwidgets import view                              
from aicssegmentation.core.visual import single_fluorescent_view
from aicsimageio.writers import OmeTiffWriter
import skimage.io
import skimage
from skimage.measure import label, regionprops
from skimage.segmentation import find_boundaries
from skimage.morphology import convex_hull_image

import numpy as np
import scipy
import math
import csv
import glob
#import nbimporter
import os
from scipy import ndimage
from matplotlib import pyplot as plt

## Functions


In [ ]:
def ConvertToMicrons(coordinates):
    """
    INPUTS:
       coordinates: single z,y,x coordinate of a point
    
    RETURNS the coordinates of the single point, in microns, in the format [x,y,z].
    """
    #Coordinates must be as [z,y,x]
    coordinateZ = coordinates[0]*0.5
    coordinateY = coordinates[1]*0.127
    coordinateX = coordinates[2]*0.127
    
    return [coordinateX, coordinateY, coordinateZ]

In [ ]:
"""
Both functions compute the real distance (IN MICRONS!) between two pixels in am image. The difference between the 
two is the order of the x,y,z values of each imput point. This is due to some functions outputing the coordinates as 
[z,y,x] and others as [z,x,y].
    
Does so by using the corresponding physical size of each pixel
"""
def ComputeDistance(centerA, centerB):
    """
    INPUTS:
       centerA: list containing the z,y,x coordinates of first point
       centerB: list containing the z,y,x coordinates of second point
    
    RETURNS the distance, in microns, between the two points
    """
    diffPixelX = centerA[2] - centerB[2]
    diffPixelY = centerA[1] - centerB[1]
    diffPixelZ = centerA[0] - centerB[0]

    return math.sqrt(
    (0.127 * diffPixelX) ** 2
    + (0.127 * diffPixelY) ** 2
    + (0.5 * diffPixelZ) ** 2)

def ComputeDistance2(centerA, centerB):
    """
    INPUTS:
       centerA: list containing the z,x,y coordinates of first point
       centerB: list containing the z,x,y coordinates of second point
    
    RETURNS the distance, in microns, between the two points
    """
    diffPixelX = centerA[1] - centerB[1]
    diffPixelY = centerA[2] - centerB[2]
    diffPixelZ = centerA[0] - centerB[0]

    return math.sqrt(
    (0.127 * diffPixelX) ** 2
    + (0.127 * diffPixelY) ** 2
    + (0.5 * diffPixelZ) ** 2)

def Average(lst):
    return sum(lst) / len(lst)

In [ ]:
def MatchObjects(imageA, imageB, maximalDistanceBetweenFrames, labelCounter):
    
    """
    A function that takes two 3D stacks and relabels the objects in the SECOND stacks in such a way that an object in the first 
    stack that is very close in space (threshold determined by maximalDistanceBetweenFrames) to an object in the second stack 
    will be labeled with the same unique ID. Any objects that newly appear in the second stack will be given a new unique ID 
    and the ID of any objects that dissapear will not be used again.
    
    The purpose of this function is to label single objects (centrosomes and nuclei) with the same label over time. Since a 
    centrosome or nucleus moves little between timepoints, a threshold distance is used to determine whether an object in one 
    timepoint (one stack) is the same object in the next timepoint (next stack).
    
    INPUTS:
       imageA: Full directory of raw (directly from Metamorph) timelapse video
       imageB: Full directory of raw (directly from Metamorph) timelapse video
       maximalDistanceBetweenFrames: Distance, in microns, between the same object between two consecutive timeframes
       labelCounter: Integer representing the max intensity value in stack A
    
    RETURNS 
        newLabelsInB: The second 3D stack with relabeled objects - array of images. The labels correspond to the uniform 
                        intensity value of the pixel forming each object.
        labelCounter: Integer representing the max intensity value in stack B aka the make ID number
    """
    
    arrayA = skimage.io.imread(imageA)
    arrayB = skimage.io.imread(imageB)
    
    blobsInA = [] #Empty list that will store dictionaries for stack A containing the label/ID of an object and the center pixel of that object
    blobsInB = [] #Empty list that will store dictionaries for stack B containing the label/ID of an object and the center pixel of that object
    newLabelsInB = np.zeros_like(arrayB) # Stores the relabeled copy of 3D stack B

    PropertiesA = skimage.measure.regionprops(arrayA) #Stores properties on each object found in stack A
    PropertiesB = skimage.measure.regionprops(arrayB) #Stores properties on each object found in stack B
    
    for propsA in PropertiesA:
        label = propsA.label
        centroid = propsA.centroid
        blobsInA.append({ "label": label, "center": centroid })
    
    for propsB in PropertiesB:
        label = propsB.label
        centroid = propsB.centroid
        blobsInB.append({ "label": label, "center": centroid })  
    
    while len(blobsInA) > 0:
        # 1. Fix blob a in A
        blobA = blobsInA[0]
        
        # 2. Find closest blob b in B to fixed blobA
        bestBlobInB = -1
        bestDistanceToBlobInB = np.inf
        for blobB in blobsInB:
            distance = ComputeDistance(blobA["center"], blobB["center"])
            if distance < bestDistanceToBlobInB:
                bestBlobInB = blobB["label"]
                bestDistanceToBlobInB = distance
            
        if bestDistanceToBlobInB < maximalDistanceBetweenFrames:
        # 3. Pair blobA with bestBlobInB by changing all pixel of B corresponding to the blobA with label of blobA
            newLabelsInB[arrayB == bestBlobInB] = blobA["label"]

        # 4. Remove the paired blobs from the lists by using
            blobsInA = list(filter(lambda i: i["label"] != blobA["label"], blobsInA))
            blobsInB = list(filter(lambda i: i["label"] != bestBlobInB, blobsInB))
            
        if bestDistanceToBlobInB > maximalDistanceBetweenFrames:
            blobsInA = list(filter(lambda i: i["label"] != blobA["label"], blobsInA))
            
    #In the scenario where there are more objects in B than in A, just relabel objects in B with higher integer    
    for blobB in blobsInB:
        oldBlobBLabel = blobB["label"]
        labelCounter = labelCounter + 1
        newLabelsInB[arrayB == oldBlobBLabel] = labelCounter
    
    return newLabelsInB, labelCounter


In [ ]:
def ComputeNucleoliRadii(imageStack, labels):
    """
    A function that computes the radius of a given labeled object in an array representing a 3D stack. 
    This function computes the volume of the sphere by counting the number of pixels, multiplying that number by the "real" 
    volume per pixel in microns^3 and computes the radius from the volume, assuming each object is a sphere.
    
    INPUTS:
       imageStack: Array of images corresponding to a 3D stack of a single timepoint of a timelapse.
       label: The ID (pixel intensity) of the specific object of interest.
    
    RETURNS 
        realRadius: Value, in microns, corresponding to the radius of the given object.
    """
    radii = list()
    if isinstance(labels, int):
        singleObject = np.zeros_like(imageStack)
        singleObject[imageStack == labels] = labels
    
        volume = np.count_nonzero(singleObject)*0.127*0.127*0.5
        radius = math.pow((3*volume)/(4*math.pi), (1.0/3))
        radii.append(radius)
    
    if isinstance(labels, list):
        for label in labels:
            singleObject = np.zeros_like(imageStack)
            singleObject[imageStack == label] = label

            volume = np.count_nonzero(singleObject)*0.127*0.127*0.5
            radius = math.pow((3*volume)/(4*math.pi), (1.0/3))
            radii.append(radius)
        
    return radii

In [ ]:
def ComputeNucleoliToNucleusAssociations(imageNuclei, imageNucleoli):
    nucleoliToNucleiAssociations = dict()
    propsNucleoli = skimage.measure.regionprops(imageNucleoli)
    propsNuclei = skimage.measure.regionprops(imageNuclei)
    
    for propsA in propsNucleoli:
        centroidA = propsA.centroid
        closestNucleus = 0
        closestDistance = float("inf")
        for propsB in propsNuclei:
            centroidB = propsB.centroid
            distance = ComputeDistance(centroidA, centroidB)
            if distance < closestDistance:
                closestNucleus = propsB.label
                closestDistance = distance
        if closestNucleus in nucleoliToNucleiAssociations:
            if isinstance(nucleoliToNucleiAssociations[closestNucleus], list):
                Newlist = nucleoliToNucleiAssociations[closestNucleus]
                Newlist.append(propsA.label)
                nucleoliToNucleiAssociations[closestNucleus] = Newlist
            if isinstance(nucleoliToNucleiAssociations[closestNucleus], int):
                nucleoliToNucleiAssociations[closestNucleus] = [nucleoliToNucleiAssociations[closestNucleus], propsA.label]

        if closestNucleus not in nucleoliToNucleiAssociations:   
            nucleoliToNucleiAssociations[closestNucleus] = propsA.label

    return nucleoliToNucleiAssociations

<span style='color:red'> **2. Change the variable `directoryNucleoli` to the same directory of the variable `directoryOUT1` found in the `dots_image_segmenter_SZ_nucleolus` jupyter notebook and add a `*` at the end. Change the variable `directoryNuclei` to the same directory of the variable `directoryOUT2` found in the `dots_image_segmenter_SZ_nucleolus` jupyter notebook and add a `*` at the end.** 

In [ ]:
directoryNucleoli = 'E:\\Weber Lab\\Practice Coding\\seg_batch_output_nucleoli\\*'
directoryNuclei = 'E:\\Weber Lab\\Practice Coding\\seg_batch_output_nuclei\\*'

## Relabeling objects in nuclei images so that same nucleus over time has the same label

<span style='color:red'> **3. Run the next cell** </span>

In [ ]:
#To get relabeled CENTROSOME images for all time points for all files:
maxLabelPossible = 0
for folder in glob.glob(directoryNuclei, recursive=True):
    numberOfTimestamps = 0
    for file_name in glob.glob(''+str(folder)+'\\*', recursive=True):
        numberOfTimestamps += 1
        
    imageA = skimage.io.imread(''+str(file_name[0:-7])+str(f"{0:02d}")+'.tiff')
    labelCounter = np.amax(imageA)
    
    for time in range(0, numberOfTimestamps - 1):
        imageA = ''+str(file_name[0:-7])+str(f"{time:02d}")+'.tiff'
        imageB = ''+str(file_name[0:-7])+str(f"{time+1:02d}")+'.tiff'
        print(imageA)
        print(imageB)
        
        maximalDistanceBetweenFrames = 0.2*20
        newLabeledImageB, Counter = MatchObjects(imageA, imageB, maximalDistanceBetweenFrames, labelCounter)
        print(labelCounter)
        print(Counter)
        if Counter>labelCounter:
            labelCounter = Counter 
        os.remove(''+str(file_name[0:-7])+str(f"{time+1:02d}")+'.tiff')
        out=newLabeledImageB.astype(np.uint8)
        writer = OmeTiffWriter()
        writer.save(out, ''+str(file_name[0:-7])+str(f"{time+1:02d}")+'.tiff')
    if labelCounter > maxLabelPossible:
        maxLabelPossible = labelCounter
        print(maxLabelPossible)

## Merging both channels together over time back into a timelapse video
This cell will allow to view the 3D stacks of both channels as a timelapse in FIJI with the correct object labels to allow you to manually select which nuclei (and their corresponding nucleoli) you want to obtain mesurements for.

    
<span style='color:red'> **4. Run the next 2 cell** </span>


In [ ]:
# Creates a folder name Paired_Nuc_Cent in the same location as your seg_batch folders
if not os.path.exists(''+str(directoryNuclei[0:-25])+str('Paired_Nuc_Cent')):
        os.makedirs(''+str(directoryNuclei[0:-25])+str('Paired_Nuc_Cent'))

In [ ]:
#Creates and saves a single files with all the timepoints of both segmented channels
dirPaired_Nuc_Cent = ''+str(directoryNuclei[0:-25])+str('Paired_Nuc_Cent')+"\\"
start = len(directoryNuclei) - 1
for (nucleolusFolder, nucleusFolder) in zip(glob.glob(directoryNucleoli), glob.glob(directoryNuclei)):
    numberOfTimestamps = 0
    for (nucleolusFileName, nucleusFileName) in zip(glob.glob(nucleolusFolder+'\\*'), glob.glob(nucleusFolder+'\\*')): 
        numberOfTimestamps += 1
    
    #first 3D image of nucleus
    allTimePoints = skimage.io.imread(''+str(nucleusFileName[0:-7])+str(f"{0:02d}")+'.tiff')
    #first 3D image of centrosome
    nucleoli = skimage.io.imread(''+str(nucleolusFileName[0:-7])+str(f"{0:02d}")+'.tiff')
    #Merge the first 3D nucleus image with the first 3D centrosome image
    allTimePoints = np.vstack((allTimePoints, nucleoli))
    #This stores the 3D images across time so when converting to hyperstack = XYZCT
    for time in range(1, numberOfTimestamps):
        nucleoli = skimage.io.imread(''+str(nucleolusFileName[0:-7])+str(f"{time:02d}")+'.tiff')
        nucleus = skimage.io.imread(''+str(nucleusFileName[0:-7])+str(f"{time:02d}")+'.tiff')
        allTimePoints = np.vstack((allTimePoints, nucleus, nucleoli))
    #Saves array with both channels across time points, merged together, into the Paired_Nuc_Cent folder.
    out=allTimePoints.astype(np.uint8)
    writer = OmeTiffWriter()
    writer.save(out, dirPaired_Nuc_Cent +str(nucleusFileName[start:-8])+'.tiff')

## Visualizing, in FIJI, both channels as a timelapse file
Now you must manually pair any two centrosomes found in the timelapse to be associated with a nucleus. Presently, we only compute the distances for centrosomes where both centrosomes are present with the nucleus.

<span style='color:red'> **5. Open one of the timelapse files from the `Paired_Nuc_Cent` folder in FIJI. A gui will pop up: unckeck all boxes and select Hyperstack under `View Stack with:`. Press OK.**

<span style='color:red'> **6. Go to `Image` > `Hyperstacks` > `Stack to Hyperstack` and select xyzct under `Order` and enter the number of channels, planes and frames and set the `Display mode` to Color.**
    
<span style='color:red'> **7. Go to `Image` > `Stacks` > `Z project` and select Max intensity under `Projection type` and press OK.**
    
<span style='color:red'> **8. Go to `Image` > `Color` > `Merge Channels...` and select C1 and C2 to be the MAX intensity projection file. A gui will pop up and with it you can change the visual colour of the channel so that nuclei are green and centrosomes are red**
    
<span style='color:red'> **9. Go to `Image` > `Adjust` > `Brightness/Contrast` and set the maximum intensity slider to zero for both colour channels.**
    
<span style='color:red'> **10. Notice that when you run your cursor over a centrosome or nucleus (depending on which chanel you have open), the pixel intensity will change for each object, the pixel value intensity is the object ID. Therefore, go through the timelapse and find the nuclei that interest you (e.g. 8-cell stage nuclei). Take note of the nuclei IDs.** <span style='color:black'> 
   

## Putting nuclei labels into csv file
<span style='color:red'> **11. Run the next cell.This will create a csv file where you can store the nuclei IDs you took note of.**

<span style='color:red'> **12. Open Pairing Labels.csv file manually. Each timelapse will already have its designated row
under `File_Name`. You need to enter the nuclei IDs, separated by commas, under `Nuclei_Labels`, for each timelapse
(e.g. 5, 6, 7, 8).**

In [ ]:
NucleiCSV = open(''+str(directoryNuclei[0:-25])+ "Nuclei of interest.csv", 'w', newline='')
filewriter = csv.writer(NucleiCSV, delimiter=',')
filewriter.writerow(['File_Name', 'Nuclei_Labels'])
start = len(directoryNuclei) - 1
for folder in glob.glob(directoryNuclei, recursive=True):
        filewriter.writerow([str(folder[start:]), str([])])
NucleiCSV.close()

## Computing the number and the radius of droplets
<span style='color:red'> **13. Run the next cell.**

In [ ]:
NucleiCSV = open(''+str(directoryNuclei[0:-25])+ "Nuclei of interest.csv")
reader = csv.reader(NucleiCSV)
results_numNucleoli = dict()
results_diameterNucleoli = dict()
for (nucleolusFolder, nucleusFolder) in zip(glob.glob(directoryNucleoli), glob.glob(directoryNuclei)):
    file_name = nucleolusFolder[len(directoryNucleoli)-1:]
    for row in reader:
        if row[0] == file_name:
            for (nucleolusFileName, nucleusFileName) in zip(glob.glob(nucleolusFolder+'\\*'), glob.glob(nucleusFolder+'\\*')):    
                timestamp = nucleolusFileName[-7:-5]
                nucleusLabels = list(eval(row[1]))
             
                imageNucleoli = skimage.io.imread(nucleolusFileName)
                imageNuclei = skimage.io.imread(nucleusFileName)
                    
                # Compute the associated nucleus of each nucleolus
                associations = ComputeNucleoliToNucleusAssociations(imageNuclei, imageNucleoli)

                # For each label...
                for nucleusLabel in nucleusLabels:
                    if nucleusLabel in associations:
                        #print("Nucleus label:", nucleusLabel)
                        if isinstance(associations[nucleusLabel], list):
                            numNucleoli = len(associations[nucleusLabel])
                        if isinstance(associations[nucleusLabel], int):
                            numNucleoli = 1

                        # ...compute the list of nucleoli diameters
                        #print("Nucleoli labels:", associations[nucleusLabel])
                        nucleoliDiameters = ComputeNucleoliRadii(imageNucleoli, associations[nucleusLabel])

                        # ...store the results
                        if (file_name, nucleusLabel) in results_numNucleoli:
                            if isinstance(results_numNucleoli[(file_name, nucleusLabel)], list):
                                NewList = results_numNucleoli[(file_name, nucleusLabel)]
                                NewList.append(numNucleoli)
                                results_numNucleoli[(file_name, nucleusLabel)] = NewList
                                
                            if isinstance(results_numNucleoli[(file_name, nucleusLabel)], int):
                                results_numNucleoli[(file_name, nucleusLabel)] = [results_numNucleoli[(file_name, nucleusLabel)], numNucleoli]
                        
                        if (file_name, nucleusLabel) not in results_numNucleoli:
                            results_numNucleoli[(file_name, nucleusLabel)] = numNucleoli
                            
                            
                        if (file_name, nucleusLabel) in results_diameterNucleoli:
                            listD = results_diameterNucleoli[(file_name, nucleusLabel)]
                            listD.append(nucleoliDiameters)
                            results_diameterNucleoli[(file_name, nucleusLabel)] = listD
                        
                        if (file_name, nucleusLabel) not in results_diameterNucleoli:
                            results_diameterNucleoli[(file_name, nucleusLabel)] = [nucleoliDiameters]
       
                        #results_numNucleoli[(file_name, nucleusLabel, timestamp)] = numNucleoli
                        #results_diameterNucleoli[(file_name, nucleusLabel, timestamp)] = nucleoliDiameters


## Graphs of the number of nucleoli per nucleus over time
<span style='color:red'> **14. Run the next cell.**

In [ ]:
for nucleus in results_numNucleoli:
    nucleoliNumbers = results_numNucleoli[nucleus]
    timepoints = []
    for timept in range(0, len(nucleoliNumbers)):
        timepoints.append(timept)
        
    import matplotlib.pyplot as plt
    plt.figure()
    colors = plt.get_cmap('cool')

    plt.plot(timepoints, nucleoliNumbers, label=f"Index = {nucleus}")
    plt.title(str(nucleus))
    plt.xlabel('Time')
    plt.ylabel('Number of nucleoli')
plt.show()

## Graphs of the average radius of nucleoli per nucleus over time
<span style='color:red'> **15. Run the next cell.**

In [ ]:
for nucleus in results_diameterNucleoli:
    allTimepts = results_diameterNucleoli[nucleus]
    AverageDiameters = []
    for nucleoliDiameters in allTimepts:
        if len(nucleoliDiameters) > 1:
            AverageDiameter = Average(nucleoliDiameters)
            AverageDiameters.append(AverageDiameter)
        if len(nucleoliDiameters)== 1:
            AverageDiameters.append(nucleoliDiameters[0])
    
    timepoints = []
    for timept in range(0, len(AverageDiameters)):
        timepoints.append(timept)
            
    import matplotlib.pyplot as plt
    plt.figure()
    colors = plt.get_cmap('cool')
    plt.plot(timepoints, AverageDiameters, label=f"Index = {nucleus}")
    plt.title(str(nucleus))
plt.show()